In [122]:
type Op =
| Noop
| Prepare of int
| Add 
module Op =
    let ofString (line:string) =
        match line.Split(" ") with 
        
        | [| "addx"; value |] -> 
            seq {
                yield Op.Prepare (value |> int)
                yield Op.Add
            }
        | [| "noop" |] | _ ->  Seq.singleton Op.Noop
    
    let convert (lines:string seq) =
        lines |> Seq.map ofString |> Seq.concat

    type State = { sum: int; to_add: int }
    let execute (ops: Op seq)=
        ops
        |> Seq.mapFold(fun (state:State) op -> 
            let newState =
                match op with
                | Op.Noop -> state
                | Op.Prepare value -> { state with to_add = value }
                | Op.Add -> { sum = state.sum + state.to_add; to_add = 0 }
            state.sum, newState
        ) { sum = 1; to_add = 0 }
        |> fst

module Signal =
    let powerAt (idx:int) (ops:int seq) =
        ops |> Seq.item (idx - 2) |> (*) idx

    let signalSum values =
        seq {
            yield values |> powerAt 20
            yield values |> powerAt 60
            yield values |> powerAt 100
            yield values |> powerAt 140
            yield values |> powerAt 180
            yield values |> powerAt 220
        }

let ResolutionFolder = __SOURCE_DIRECTORY__
let lines = File.ReadLines(ResolutionFolder + "/input10.txt")

let values =
    lines
    |> Op.convert
    |> Op.execute

values |> Signal.signalSum |> Seq.sum

14300

In [123]:
module CRT =
    let byLine (values:int seq) =
        values |> Seq.chunkBySize 40
    
    // ...*...
    // .#####.
    // 0123456
    let pixel (column:int) (signal:int)=
        let offset = Math.Abs(column - signal)

        

        if offset <= 1 then
            "🟡"
        else
            "⚫"
    let drawLine (values:int seq) =
        values 
        |> Seq.mapi pixel
        |> Seq.reduce (+)

    let drawScreen (values: int seq) =
        values 
        |> byLine 
        |> Seq.map drawLine

values |> CRT.drawScreen

index,value
0,🟡🟡⚫⚫🟡🟡⚫⚫🟡🟡⚫⚫🟡🟡⚫⚫🟡🟡⚫⚫🟡🟡⚫⚫🟡🟡⚫⚫🟡🟡⚫⚫🟡🟡⚫⚫🟡🟡⚫⚫
1,🟡🟡🟡⚫⚫⚫🟡🟡🟡⚫⚫⚫🟡🟡🟡⚫⚫⚫🟡🟡🟡⚫⚫⚫🟡🟡🟡⚫⚫⚫🟡🟡🟡⚫⚫⚫🟡🟡🟡⚫
2,🟡🟡🟡🟡⚫⚫⚫⚫🟡🟡🟡🟡⚫⚫⚫⚫🟡🟡🟡🟡⚫⚫⚫⚫🟡🟡🟡🟡⚫⚫⚫⚫🟡🟡🟡🟡⚫⚫⚫⚫
3,🟡🟡🟡🟡🟡⚫⚫⚫⚫⚫🟡🟡🟡🟡🟡⚫⚫⚫⚫⚫🟡🟡🟡🟡🟡⚫⚫⚫⚫⚫🟡🟡🟡🟡🟡⚫⚫⚫⚫⚫
4,🟡🟡🟡🟡🟡🟡⚫⚫⚫⚫⚫⚫🟡🟡🟡🟡🟡🟡⚫⚫⚫⚫⚫⚫🟡🟡🟡🟡🟡🟡⚫⚫⚫⚫⚫⚫🟡🟡🟡🟡
5,🟡🟡🟡🟡🟡🟡🟡⚫⚫⚫⚫⚫⚫⚫🟡🟡🟡🟡🟡🟡🟡⚫⚫⚫⚫⚫⚫⚫🟡🟡🟡🟡🟡🟡🟡⚫⚫⚫⚫⚫
